# 1. EDA
En este apartado se pretende analizar los datasets para poder enfocar mejor la limpieza

## 1.1 Importación y carga de datos
Debemos declarar las librerías que usamos y leer el correspondiente archivo de datos

In [2]:
# Importación de librerías
import pandas as pd

# Lectura dataset separado por caracteres ';'
df = pd.read_csv('../estaciones_meteo_CodigoPostal.csv', sep=';')

## 1.2 Configuración de Pandas
Para poder leer bien los resultados de las ejecuciones, vamos a configurar tanto el número máximo de columnas como el número máximo de filas

In [3]:
# Número máximo de filas a mostrar
pd.set_option('display.max_rows', None)

# Número máximo de columnas a mostrar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 2000)

## 1.3 Descripción general del dataset
Para poder conocer ciertas características relevantes del dataset, como el número de instancias (filas) y características (columnas) procederemos a usar diferentes funciones de Pandas


In [4]:
# Descripción de parámetros generales -> count, mean, std, min, 25%, 50%, 75%, max
print(df.describe())

# Número de filas y columnas del dataset
print("\n")
print("Número de filas: ", df.shape[0])
print("Número de columnas: ", df.shape[1])
print("\n")

# Para saber el tipo de variable de cada columna
print(df.info())

             CÓDIGO
count  2.600000e+01
mean   2.807907e+07
std    4.035567e+01
min    2.807900e+07
25%    2.807904e+07
50%    2.807908e+07
75%    2.807911e+07
max    2.807912e+07


Número de filas:  26
Número de columnas:  3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CÓDIGO         26 non-null     int64 
 1   DIRECCION      26 non-null     object
 2   Codigo Postal  26 non-null     object
dtypes: int64(1), object(2)
memory usage: 756.0+ bytes
None


## 1.4 Observación inicial del dataset
Vamos a mostrar todas las entradas para poder observar cómo es realmente por dentro el dataset

In [5]:
# Mostramos todo el dataset, ya que es pequeño
print(df)

      CÓDIGO                                             DIRECCION   Codigo Postal
0   28079004                                       Plaza de España  [28008, 28013]
1   28079008                        Entre C/ Alcalá y C/ O’ Donell           28009
2   28079016      C/ Arturo Soria  esq. C/  Vizconde de los Asilos           28043
3   28079018                           Calle Farolillo - C/Ervigio           28047
4   28079024              Casa de Campo  (Terminal del Teleférico)           28011
5   28079035                    Plaza del Carmen esq. Tres Cruces.           28013
6   28079036          Avd. Moratalaz  esq. Camino de los Vinateros           28030
7   28079038           Avd. Pablo Iglesias esq. C/ Marqués de Lema           28039
8   28079039              Avd. Betanzos esq. C/  Monforte de Lemos           28034
9   28079054                     Avda La Gavia / Avda. Las Suertes           28051
10  28079056                          Pza. Elíptica - Avda. Oporto           28025
11  

## 1.5 Revisión de valores nulos
Como ya se ha visto en el anterior apartado (info), podemos observar los valores nulos de esa forma. Pero se puede observar de una forma más visual en esta sección y, además, hay que tener en cuenta valores como el cero que también pueden considerarse nulos.

In [6]:
# Para poder saber el número de valores faltantes
print("    ----------    Valores faltantes    ----------    ")
print("\n")
print(df.isnull().sum())
print("\n")

# Para poder saber el número de ceros en cada columna
print("    ----------    Valores cero    ----------    ")
print("\n")
print((df == 0).sum())

    ----------    Valores faltantes    ----------    


CÓDIGO           0
DIRECCION        0
Codigo Postal    0
dtype: int64


    ----------    Valores cero    ----------    


CÓDIGO           0
DIRECCION        0
Codigo Postal    0
dtype: int64


No encontramos valores nulos, por lo que no se hará limpieza de esto

## 1.6 Identificación de fechas no estandarizadas
Se deben identificar las fechas que no se encuentran en el formato adecuado para MongoDB (DD/MM/YYYY)

Como se puede apreciar, no existen fechas por lo que este paso se omitirá

## 1.7 Identificación de registros duplicados
Debemos validar que no existen filas iguales que ensucien el dataset, sobre todo estando pendiente de duplicaciones de la clave primaria

In [9]:
# Ver las filas duplicadas
print("   ----------    Filas duplicadas    ----------    ")
print("\n")
print(df[df.duplicated()])
print("\n")

# Número de filas duplicadas
print("Número de filas duplicadas: ", df.duplicated().sum())
print("\n")

# Filtrar filas que tienen el mismo ID
duplicados = df.groupby('CÓDIGO').filter(lambda x: len(x) > 1)

# Ordenar por NIF para que las filas con el mismo NIF se visualicen una encima de la otra
duplicados = duplicados.sort_values(by='CÓDIGO')

# Mostrar las filas con la misma PK
print("   ----------    Filas con el mismo CÓDIGO    ----------    ")
print("\n")
print(duplicados)
print("\n")
print("Número de filas con el mismo CÓDIGO: ", duplicados.shape[0])

   ----------    Filas duplicadas    ----------    


Empty DataFrame
Columns: [CÓDIGO, DIRECCION, Codigo Postal]
Index: []


Número de filas duplicadas:  0


   ----------    Filas con el mismo CÓDIGO    ----------    


Empty DataFrame
Columns: [CÓDIGO, DIRECCION, Codigo Postal]
Index: []


Número de filas con el mismo CÓDIGO:  0


No existen filas iguales, por lo que este paso de limpieza se omitirá

## 1.8 Búsqueda de errores tipográficos
Hay ciertos atributos de texto que pueden contar con determinados errores tipográficos que deben ser solucionados, como los nombres de áreas, juegos, usuarios y ubicaciones

En este dataset no encontramos ningún atributo similar, por lo que este paso no se realizará

## 1.9 Identificación de valores enum fuera de campo
Hay ciertos atributos que solo deben poseer ciertos valores (como Operativo-NoOperativo). Hace falta identificar aquellos valores de ese campo fuera de la norma

No existen campos Enum en este dataset, por lo que se omitirá este paso en la limpieza

## 1.10 Validación de las coordenadas y otros campos geoespaciales
Hay algunas veces en las que los códigos postales no respetan la identificación de Madrid (280..) o el formato, ya sean códigos postales u otros atributos de geolocalización

In [ ]:
# Muestra los códigos postales que no son válidos
print("   ----------    Códigos postales    ----------    ")
print("\n")
print(df['Codigo Postal'])

   ----------    Códigos postales    ----------    


0     [28008, 28013]
1              28009
2              28043
3              28047
4              28011
5              28013
6              28030
7              28039
8              28034
9              28051
10             28025
11             28044
12             28042
13             28025
14             28026
15             28012
16             28026
17             28043
18             28035
19             28010
20             28013
21             28002
22             28018
23             28018
24             28045
25             28045
Name: Codigo Postal, dtype: object




Todos los códigos postales corresponden a códigos existentes, por lo que no se precisa de una limpieza

## 1.11 Identificación de unidades de medida en un formato no estandarizado
Se deben identificar las filas que no posean un formato estándar

En este dataset no encontramos valores de unidades de medida, por lo que este paso no se realizará

## 1.12 Otros atributos a corregir
En esta sección se mencionarán aquellos atributos que también deban ser limpiados por errores

En esta sección, al ser el dataset muy pequeño y verse claramente que no se necesita validar los parámetros, se va a saltar

# 2. Limpieza de los datasets
En este apartado se realizará la limpieza según la información obtenida en el análisis exploratorio de datos:
- No se precisa la limpieza de ningún campo

Aun así, nos conviene transformar los datos de tal forma que la importación de información al dataset meteo24.csv sea más sencilla:
- Asumir que plaza de España solo posee el código postal 28008 ya que encontramos otro 280013 en los códigos postales
  - Esto ayuda a facilitar la asignación de distritos
- Crear una columna de distritos para que la importación de la columna DISTRITO a meteo24.csv sea más sencilla

## 2.1 Eliminación de código postal 280013 a Plaza España
Debemos eliminar el código postal 280013 a Plaza España para facilitar la asignación de distritos

In [16]:
# Encuentra la fila con la dirección 'Plaza de España'
print("   ----------    Filas con la dirección 'Plaza de España'    ----------    ")
print("\n")
print(df.loc[(df['DIRECCION'] == 'Plaza de España')])
print("\n")

# Modifica el valor de esa fila de la columna Codigo Postal a 28008
print("   ----------    Modificación de la dirección 'Plaza de España'    ----------    ")
print("\n")
df.loc[(df['DIRECCION'] == 'Plaza de España'), 'Codigo Postal'] = 28008
print(df.loc[(df['DIRECCION'] == 'Plaza de España')])


   ----------    Filas con la dirección 'Plaza de España'    ----------    


     CÓDIGO        DIRECCION   Codigo Postal
0  28079004  Plaza de España  [28008, 28013]


   ----------    Modificación de la dirección 'Plaza de España'    ----------    


     CÓDIGO        DIRECCION Codigo Postal
0  28079004  Plaza de España         28008


## 2.2 Creación de la columna DISTRITO
Se va a crear esta columna para poder asociar los distritos más fácilmente a cada estación de meteorología

NOTA: Los códigos postales son erróneos. Mirando una web del ayuntamiento de Madrid, se puede apreciar como los códigos postales no son los que mencionan aquí, por lo que este paso se ha hecho de manera manual 
(web: https://www.madrid.es/portales/munimadrid/es/Inicio/Medio-ambiente/Direcciones-y-telefonos/Red-de-meteorologia/?vgnextfmt=default&vgnextoid=b84e80670b4d4710VgnVCM1000001d4a900aRCRD&vgnextchannel=864f79ed268fe410VgnVCM1000000b205a0aRCRD)

NOTA 2: Hay una fila ('Embajadores') que, contrastándola con la web oficial de Madrid, no existe. Se ha asumido que es un error y que se refiere al centro de metereología 'Aguas La China', el único que quedaba libre, que se encuentra en Puente de Vallecas

In [17]:
# Creamos una columna nueva para los distritos
print("   ----------    Creación de la columna 'DISTRITO'    ----------    ")
print("\n")

# Manualmente (accediendo a una web del ayuntamiento de Madrid) creamos una lista con los distritos según su dirección
distritos = ['Moncloa-Aravaca', 'Salamanca', 'Ciudad Lineal', 'Carabanchel', 'Moncloa-Aravaca', 'Centro', 'Moratalaz', 'Chamberí', 'Fuencarral-El Pardo', 'Villa de Vallecas', 'Carabanchel', 'Fuencarral-El Pardo', 'Barajas', 'Moratalaz', 'Villaverde', 'Puente de Vallecas', 'Moncloa-Aravaca', 'Hortaleza', 'Fuencarral-El Pardo', 'Chamberí', 'Centro', 'Chamartín', 'Puente de Vallecas', 'Puente de Vallecas', 'Arganzuela', 'Arganzuela']

# Añadimos la columna al Dataset
df['DISTRITO'] = distritos

# Mostramos el Dataset con la nueva columna
print(df)

   ----------    Creación de la columna 'DISTRITO'    ----------    


      CÓDIGO                                             DIRECCION Codigo Postal             DISTRITO
0   28079004                                       Plaza de España         28008      Moncloa-Aravaca
1   28079008                        Entre C/ Alcalá y C/ O’ Donell         28009            Salamanca
2   28079016      C/ Arturo Soria  esq. C/  Vizconde de los Asilos         28043        Ciudad Lineal
3   28079018                           Calle Farolillo - C/Ervigio         28047          Carabanchel
4   28079024              Casa de Campo  (Terminal del Teleférico)         28011      Moncloa-Aravaca
5   28079035                    Plaza del Carmen esq. Tres Cruces.         28013               Centro
6   28079036          Avd. Moratalaz  esq. Camino de los Vinateros         28030            Moratalaz
7   28079038           Avd. Pablo Iglesias esq. C/ Marqués de Lema         28039             Chamberí
8   2807903

## 2.4 Mostrar Dataset Limpio y guardar CSV
Vamos a mostrar algunas filas del dataset limpio para validar que todo está OK y guardamos el Dataset

In [ ]:
# Guardamos el dataset limpio
df.to_csv('../estaciones_meteo_CodigoPostalLimpio.csv', index=False)
print("Dataset limpio guardado")



Dataset limpio guardado
